In [95]:
! activate ai-azure-c1
# ! pip install azure-cognitiveservices-vision-customvision
import sys

sys.path.append("/opt/conda/envs/ai-azure-c1/lib/python3.8/site-packages")

import requests
from urllib.parse import urlparse
from io import BytesIO
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

import glob, os, sys, time, uuid

from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials


def show_image_in_cell(img_url):
    response = requests.get(img_url)
    img = Image.open(BytesIO(response.content))
    plt.figure(figsize=(20,10))
    plt.imshow(img)
    plt.show()

CUSTOM_VISION_ENDPOINT = "https://ai-nanodegree-p1.cognitiveservices.azure.com/"
custom_vision_key = "e7f9d136f2ac43cf9ddc17b3ce9fd7b9"
custom_vision_resource_id = '/subscriptions/59548b95-35f6-464c-9646-bd4fcf73a445/resourceGroups/ML-RG/providers/Microsoft.CognitiveServices/accounts/ai-nanodegree-p1'

/anaconda/envs/azureml_py38/bin/activate: 5: /anaconda/envs/azureml_py38/etc/conda/activate.d/activate-binutils_linux-64.sh: Syntax error: "(" unexpected


In [87]:
training_credentials = ApiKeyCredentials(in_headers={"Training-key": custom_vision_key})
trainer = CustomVisionTrainingClient(CUSTOM_VISION_ENDPOINT, training_credentials)

prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": custom_vision_key})
predictor = CustomVisionPredictionClient(CUSTOM_VISION_ENDPOINT, prediction_credentials)

# Find the object detection domain
obj_detection_domain = next(domain for domain in trainer.get_domains() if domain.type == "ObjectDetection" and domain.name == "General (compact) [S1]")

# Create a new project
#project_name = uuid.uuid4() #5a8c190f-fa61-4879-a734-2c6a3f71bb59
#project = trainer.create_project(project_name, domain_id=obj_detection_domain.id)

#using existing project with tagged images to avoid re-work
project = trainer.get_project("d2f83227-cad7-45be-92c7-03bb43bd5572")
#lighter_tag = trainer.create_tag(project.id, "lighter")

In [88]:
#if no changes since last training -> expect error
iteration = trainer.train_project(project.id)
trainer.get_iteration()
while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project.id, iteration.id)
    print ("Training status: " + iteration.status)
    print ("Waiting 10 seconds...")
    time.sleep(10)

TypeError: get_iteration() missing 2 required positional arguments: 'project_id' and 'iteration_id'

In [ ]:
iteration_list = trainer.get_iterations(project.id)
model_perf = trainer.get_iteration_performance(project.id, iteration_list[0].id)
model_perf.as_dict()

In [56]:
publish_iteration_name = "lighter_detection_python_trained"
trainer.publish_iteration(project.id, iteration.id, publish_iteration_name, custom_vision_resource_id)


True

In [96]:
project_id = "d2f83227-cad7-45be-92c7-03bb43bd5572"
publish_iteration_name = "lighter-detection-improved"

def perform_prediction_cust(image_file_name,):
    with open(image_file_name, "rb") as image_contents:
        results = predictor.detect_image(project.id, publish_iteration_name, image_contents.read())
        # Display the results.
        print("Predictions for: "+image_file_name)
        for prediction in results.predictions:
            print("\t" + prediction.tag_name +
                  ": {0:.2f}%".format(prediction.probability * 100))

In [104]:
perform_prediction_cust("lighter_test_set_3of5.jpg")

Predictions for: lighter_test_set_3of5.jpg
	lighter: 72.29%
	lighter: 2.55%
	lighter: 2.14%
	lighter: 1.47%


In [67]:
def perform_prediction(image_file_name,):
    with open(image_file_name, "rb") as image_contents:
        results = predictor.detect_image(project.id, publish_iteration_name, image_contents.read())
        # Display the results.
        print("Predictions for: "+image_file_name)
        for prediction in results.predictions:
            print("\t" + prediction.tag_name +
                  ": {0:.2f}%".format(prediction.probability * 100))

In [ ]:
lighter_images = [file for file in glob.glob('*.jpg') if file.startswith("lighter_test")]
for image in lighter_images:
    perform_prediction(image)
    ## Checking the Image
    with open(image, 'rb') as img_code:
        img_view_ready = Image.open(img_code)
        plt.figure()
        plt.imshow(img_view_ready)